In [ ]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(max_depth=100, random_state=0)
regr.fit(X_train, y_train)

y_test_pred=regr.predict(X_test)
regr.score(X_test,y_test)

0.2941381284680471

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

regr = AdaBoostRegressor(random_state=0, n_estimators=100)
regr.fit(X_train, y_train)

y_test_pred=regr.predict(X_test)
regr.score(X_test,y_test)

0.06590682071387943

In [ ]:
from sklearn.tree import DecisionTreeRegressor

regr = DecisionTreeRegressor(random_state=0)
regr.fit(X_train, y_train)

y_test_pred=regr.predict(X_test)
regr.score(X_test,y_test)

-0.994425047621623

In [ ]:
from sklearn.linear_model import LinearRegression

regr = LinearRegression()
regr.fit(X_train, y_train)

y_test_pred=regr.predict(X_test)
regr.score(X_test,y_test)

0.5337686368701332

--- 문제점 --- 정확도가 너무 낮음

1. 학습 데이터가 너무 부족
-> 학습 데이터를 늘리려면 지역을 무시하여 각 년도별 24개의 데이터 확보
-> 총 24*7 개의 데이터 확보 가능
-> 지역적 특성을 무시하는 결과..

-> 년도별 데이터를 월,일 로 쪼개면..?

2. 데이터마다 중요도에 따라 가중치를 부여해주는 작업이 필요해보임

3. 데이터마다 편차가 커서 학습 이전에 별도의 전처리가 필요해보임

4. 데이터가 적은 만큼 일반화를 위해 교차검증할 필요 있음

---  10/24 ---
  오늘은 지수 별로 시각화 하고 사이킷런을 통해 아웃풋을 이끌어 내보자

In [2]:
import numpy as np
import pandas as pd

In [90]:
df_crime=pd.read_csv('/content/drive/MyDrive/multicampus/beaconomic/5대 범죄 발생현황.csv',index_col=0) # 범죄 -> 위해지표
df_cctv=pd.read_csv('/content/drive/MyDrive/multicampus/beaconomic/CCTV_전처리_최종.csv', encoding='CP949',index_col=0) # CCTV -> 경감지표
df_light=pd.read_csv('/content/drive/MyDrive/multicampus/beaconomic/street_light.csv', encoding='CP949',index_col=0) # 가로등 -> 경감지표
df_police=pd.read_csv('/content/drive/MyDrive/multicampus/beaconomic/경찰서 및 파출소 수 최종.csv',index_col=0) # 경찰서수 -> 경감지표
df_female=pd.read_csv('/content/drive/MyDrive/multicampus/beaconomic/여성가구원최종.csv',index_col=0) # 여성 수 -> 취약지표
df_dense=pd.read_csv('/content/drive/MyDrive/multicampus/beaconomic/dense_preprocess.csv',index_col=0) # 취약지표
df_112=pd.read_excel('/content/drive/MyDrive/multicampus/beaconomic/지역구별 112 신고건수_전처리.xlsx',index_col=0) # 신고건수

In [134]:
df_crime

,2015,2016,2017,2018,2019,2020,2021
자치구별(2),,,,,,,
강남구,8617,8149,7720,7513,7304,7356,6146
강동구,5244,4462,4261,3919,4014,3788,3458
강북구,4257,4229,3393,3437,3838,2770,2301
강서구,5585,5450,5135,4629,4924,4415,3896
관악구,6345,5678,5525,5026,5328,5261,4444
광진구,5909,5322,4646,3915,4011,3601,3087
구로구,5646,5366,4895,4810,4707,4175,3486
금천구,3781,3645,3265,3293,3105,2598,2439
노원구,5130,4543,4209,4007,4153,3743,3425


In [91]:
df_crime=df_crime.drop('Unnamed: 2',axis=1)

In [92]:
df_crime=df_crime.set_index('자치구별(2)')

In [93]:
import re

def only_numders(string) :
  string=re.sub(r'[^0-9]', '', string)
  return string

In [94]:
for c in df_female.columns:
  df_female[c]=df_female[c].map(only_numders)

In [95]:
l=list(df_112.index)

In [96]:
for i in range(0,len(l)):
  if (l[i]!='중구'):
    l[i]=l[i]+'구'

In [97]:
df_112.set_index([l],inplace=True) # 추가 전처리

In [124]:
df_total=pd.DataFrame(columns=['범죄현황','CCTV','가로등','경찰서','여성인구','인구밀도','112신고건수'])

In [126]:
df_112.columns

Index(['2015', '2016', '2017', '2018', '2019', '2020', '2021'], dtype='object')

In [129]:
df_112.index

'강남구'

In [128]:
df_total.columns

Index(['범죄현황', 'CCTV', '가로등', '경찰서', '여성인구', '인구밀도', '112신고건수'], dtype='object')

In [130]:
df_112[df_112.columns[0]][df_112.index[0]]

169303

In [125]:
# for total_col in df_total.columns:
#   #df_total[total_col]
#   if (total_col=='범죄현황'):
#     for x in df_crime.columns:
#       for y in df_crime.index:
#         df_total[total_col]=df_crime[x][y]
  
#   elif (total_col=='CCTV'):
#     for x in df_cctv.columns:
#       for y in df_cctv.index:


,범죄현황,CCTV,가로등,경찰서,여성인구,인구밀도,112신고건수


In [405]:
df_total=pd.read_csv('/content/통합.csv', encoding='CP949')

In [406]:
df_total

,5대범죄,CCTV,인구밀도,가로등,경찰서수,여성인구,112신고건수,인구
0,8617,830,14728,8988,25,"282,162",169303,581760
1,5244,154,18842,8651,20,"223,788",94193,463321
2,4257,124,14172,6977,13,"163,606",79634,334426
3,5585,199,14376,5299,16,"292,237",116561,595691
4,6345,609,17891,4981,11,"257,655",113627,529031
...,...,...,...,...,...,...,...,...
170,2381,228,10852,5789,19,"114,778",70794,237285
171,3244,471,16061,12593,21,"239,230",91838,477173
172,2712,0,6431,14351,23,"77,490",69511,153789
173,2861,403,13231,14822,22,"65,653",127597,131787


In [407]:
df_total['여성인구']=df_total['여성인구'].map(only_numders)

In [408]:
df_total

,5대범죄,CCTV,인구밀도,가로등,경찰서수,여성인구,112신고건수,인구
0,8617,830,14728,8988,25,282162,169303,581760
1,5244,154,18842,8651,20,223788,94193,463321
2,4257,124,14172,6977,13,163606,79634,334426
3,5585,199,14376,5299,16,292237,116561,595691
4,6345,609,17891,4981,11,257655,113627,529031
...,...,...,...,...,...,...,...,...
170,2381,228,10852,5789,19,114778,70794,237285
171,3244,471,16061,12593,21,239230,91838,477173
172,2712,0,6431,14351,23,77490,69511,153789
173,2861,403,13231,14822,22,65653,127597,131787


In [409]:
X=df_total.drop('112신고건수',axis=1)
y=df_total['112신고건수']

In [410]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [411]:
from sklearn.model_selection import cross_val_score

In [412]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(max_depth=100, random_state=0)
regr.fit(X_train, y_train)

y_test_pred=regr.predict(X_test)
print(regr.score(X_test,y_test))

scores = cross_val_score(regr, X, y)
print(scores.mean())

0.8551138324360155
0.7871656862532177


In [413]:
from sklearn.linear_model import LinearRegression

regr = LinearRegression()
regr.fit(X_train, y_train)

y_test_pred=regr.predict(X_test)
print(regr.score(X_test,y_test))

scores = cross_val_score(regr, X, y)
print(scores.mean())

0.4813831349136145
0.42379440063179324


In [414]:
from sklearn.tree import DecisionTreeRegressor

regr = DecisionTreeRegressor(random_state=0)
regr.fit(X_train, y_train)

y_test_pred=regr.predict(X_test)
print(regr.score(X_test,y_test))

scores = cross_val_score(regr, X, y)
print(scores.mean())

0.6125243270070375
0.6779929569145245


In [415]:
from sklearn.ensemble import AdaBoostRegressor

regr = AdaBoostRegressor(random_state=0, n_estimators=100)
regr.fit(X_train, y_train)

y_test_pred=regr.predict(X_test)
print(regr.score(X_test,y_test))

scores = cross_val_score(regr, X, y)
print(scores.mean())

0.8411066178606017
0.816778334895018


In [416]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [417]:
X_train.head()

,5대범죄,CCTV,인구밀도,가로등,경찰서수,여성인구,인구
96,3590,635,16415,12691,21,239100,487666
68,3882,577,27289,9209,16,229608,475018
168,3015,371,25882,9653,16,221403,450487
11,4074,128,25240,6835,22,207292,412774
97,3690,85,6817,14193,25,80820,163026


In [418]:
X_train_std=X_train

In [419]:
X_train_std.head()

,5대범죄,CCTV,인구밀도,가로등,경찰서수,여성인구,인구
96,3590,635,16415,12691,21,239100,487666
68,3882,577,27289,9209,16,229608,475018
168,3015,371,25882,9653,16,221403,450487
11,4074,128,25240,6835,22,207292,412774
97,3690,85,6817,14193,25,80820,163026


In [420]:
X_train_std.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117 entries, 96 to 102
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   5대범죄    117 non-null    int64 
 1   CCTV    117 non-null    int64 
 2   인구밀도    117 non-null    int64 
 3   가로등     117 non-null    int64 
 4   경찰서수    117 non-null    int64 
 5   여성인구    117 non-null    object
 6   인구      117 non-null    int64 
dtypes: int64(6), object(1)
memory usage: 7.3+ KB


In [421]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117 entries, 96 to 102
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   5대범죄    117 non-null    int64 
 1   CCTV    117 non-null    int64 
 2   인구밀도    117 non-null    int64 
 3   가로등     117 non-null    int64 
 4   경찰서수    117 non-null    int64 
 5   여성인구    117 non-null    object
 6   인구      117 non-null    int64 
dtypes: int64(6), object(1)
memory usage: 7.3+ KB


In [422]:
#sc.fit_transform([X_train_std['5대범죄']])[0].shape

In [423]:
#X_train_std['5대범죄'].shape

In [424]:
#X_train_std['5대범죄']=sc.fit_transform([X_train_std['5대범죄']])[0]

In [425]:
df_total_onehot=pd.read_csv('/content/통합_onehot.csv', encoding='CP949')

In [426]:
df_total_onehot

,5대범죄,CCTV,인구밀도,가로등,경찰서수,여성인구,112신고건수,인구,강남구,강동구,...,성동구,성북구,송파구,양천구,영등포구,용산구,은평구,종로구,중구,중랑구
0,8617,830,14728,8988,25,"282,162",169303,581760,1,0,...,0,0,0,0,0,0,0,0,0,0
1,5244,154,18842,8651,20,"223,788",94193,463321,0,1,...,0,0,0,0,0,0,0,0,0,0
2,4257,124,14172,6977,13,"163,606",79634,334426,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5585,199,14376,5299,16,"292,237",116561,595691,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6345,609,17891,4981,11,"257,655",113627,529031,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,2381,228,10852,5789,19,"114,778",70794,237285,0,0,...,0,0,0,0,0,1,0,0,0,0
171,3244,471,16061,12593,21,"239,230",91838,477173,0,0,...,0,0,0,0,0,0,1,0,0,0
172,2712,0,6431,14351,23,"77,490",69511,153789,0,0,...,0,0,0,0,0,0,0,1,0,0
173,2861,403,13231,14822,22,"65,653",127597,131787,0,0,...,0,0,0,0,0,0,0,0,1,0


In [427]:
df_total_onehot['여성인구']=df_total_onehot['여성인구'].map(only_numders)

In [428]:
X=df_total_onehot.drop('112신고건수',axis=1)
y=df_total_onehot['112신고건수']

In [429]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [430]:
from sklearn.model_selection import cross_val_score

In [431]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(max_depth=100, random_state=0)
regr.fit(X_train, y_train)

y_test_pred=regr.predict(X_test)
print(regr.score(X_test,y_test))

scores = cross_val_score(regr, X, y)
print(scores.mean())

0.9151286327015289
0.8597896081198488


In [432]:
from sklearn.linear_model import LinearRegression

regr = LinearRegression()
regr.fit(X_train, y_train)

y_test_pred=regr.predict(X_test)
print(regr.score(X_test,y_test))

scores = cross_val_score(regr, X, y)
print(scores.mean())

0.9882202279625926
0.9884395404401545


In [433]:
from sklearn.tree import DecisionTreeRegressor

regr = DecisionTreeRegressor(random_state=0)
regr.fit(X_train, y_train)

y_test_pred=regr.predict(X_test)
print(regr.score(X_test,y_test))

scores = cross_val_score(regr, X, y)
print(scores.mean())

0.9361874636618323
0.7684170277172179


In [434]:
from sklearn.ensemble import AdaBoostRegressor

regr = AdaBoostRegressor(random_state=0, n_estimators=100)
regr.fit(X_train, y_train)

y_test_pred=regr.predict(X_test)
print(regr.score(X_test,y_test))

scores = cross_val_score(regr, X, y)
print(scores.mean())

0.8406206327000445
0.8643023637379811
